## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lindon,US,40.3433,-111.7208,53.51,51,75,1.01,broken clouds
1,1,Rikitea,PF,-23.1203,-134.9692,73.29,73,99,17.25,overcast clouds
2,2,Sioux Lookout,CA,50.1001,-91.9170,68.38,77,1,6.91,clear sky
3,3,Barcelos,PT,41.5388,-8.6151,58.84,70,66,3.71,broken clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,38.86,87,75,12.66,light rain


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,73.29,73,99,17.25,overcast clouds
2,2,Sioux Lookout,CA,50.1001,-91.9170,68.38,77,1,6.91,clear sky
6,6,Bonthe,SL,7.5264,-12.5050,75.87,88,87,5.48,overcast clouds
9,9,Kamaishi,JP,39.2667,141.8833,77.95,77,32,15.55,scattered clouds
13,13,Carutapera,BR,-1.1950,-46.0200,76.53,88,36,5.12,light rain
16,16,Butaritari,KI,3.0707,172.7902,82.71,74,62,15.50,light rain
18,18,Carnarvon,AU,-24.8667,113.6333,78.87,57,40,11.50,scattered clouds
20,20,Komsomolskiy,UZ,40.4272,71.7189,66.92,37,0,4.61,clear sky
23,23,Atuona,PF,-9.8000,-139.0333,77.90,76,17,19.57,light rain
27,27,Araguacu,BR,-12.9306,-49.8264,81.99,51,44,3.83,scattered clouds


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                362
City                   362
Country                360
Lat                    362
Lng                    362
Max Temp               362
Humidity               362
Cloudiness             362
Wind Speed             362
Current Description    362
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID                360
City                   360
Country                360
Lat                    360
Lng                    360
Max Temp               360
Humidity               360
Cloudiness             360
Wind Speed             360
Current Description    360
dtype: int64

In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.count()

City                   360
Country                360
Max Temp               360
Current Description    360
Lat                    360
Lng                    360
Hotel Name             360
dtype: int64

In [27]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found in", row['City'], ".. skipping.")
        hotel_df.loc[index, "Hotel Name"] = "NA"

Hotel not found in Wagar .. skipping.
Hotel not found in Poya .. skipping.
Hotel not found in Kidal .. skipping.
Hotel not found in Goure .. skipping.
Hotel not found in Mirabad .. skipping.
Hotel not found in Taoudenni .. skipping.
Hotel not found in Birao .. skipping.
Hotel not found in Aitape .. skipping.
Hotel not found in Nalut .. skipping.
Hotel not found in Avera .. skipping.
Hotel not found in Khasan .. skipping.
Hotel not found in Adrar .. skipping.
Hotel not found in Marabba .. skipping.
Hotel not found in Awjilah .. skipping.
Hotel not found in Sabla .. skipping.
Hotel not found in Yaqui .. skipping.
Hotel not found in Dekar .. skipping.
Hotel not found in Tessalit .. skipping.
Hotel not found in Yaan .. skipping.
Hotel not found in Mundo Nuevo .. skipping.
Hotel not found in Minador Do Negrao .. skipping.
Hotel not found in Tiznit .. skipping.
Hotel not found in Nioro .. skipping.
Hotel not found in Gushi .. skipping.
Hotel not found in Amapa .. skipping.
Hotel not found in

In [33]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df=hotel_df[hotel_df["Hotel Name"] != "NA"]
pd.options.display.float_format = "{:,.2f}".format
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,73.29,overcast clouds,-23.12,-134.97,People ThankYou
2,Sioux Lookout,CA,68.38,clear sky,50.10,-91.92,Knobby's Fly-In Camps
6,Bonthe,SL,75.87,overcast clouds,7.53,-12.51,Bonthe Holiday Village
9,Kamaishi,JP,77.95,scattered clouds,39.27,141.88,Hotel Route Inn Kamaishi
13,Carutapera,BR,76.53,light rain,-1.20,-46.02,Lidera
...,...,...,...,...,...,...,...
707,Aguas Verdes,PE,69.98,overcast clouds,-3.48,-80.25,Sol Del Sur Hotel
709,Jacksonville,US,75.97,mist,30.33,-81.66,Hyatt Regency Jacksonville Riverfront
710,Labuhan,ID,89.37,clear sky,-6.88,112.21,PT PPI Lamongan Tambak C
711,Itarema,BR,78.46,scattered clouds,-2.92,-39.92,"Sea Lounge! Bar, Club & Restaurant"


In [35]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [47]:
# 11a. Add a marker layer for each city to the map. 

markers_location = locations
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))